In [1]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Building the dataframe with the results

In [2]:
datapath = '../data/img_labeling_2nd_round/'

In [3]:
label_files = sorted(os.listdir(datapath))
label_files = [f for f in label_files if 'task_labeling' in f]
for f in label_files:
    print(f)

1st_task_labeling_Amelie_2020-11-04-07-41-41.csv
1st_task_labeling_Gerda_2020-10-23-12-56-45.csv
1st_task_labeling_Marcos_2020-11-09-04-46-17.csv
1st_task_labeling_Renato_2020-11-06.csv
1st_task_labeling_Yalemisew_2020-11-02-01-40-27.csv
2nd_task_labeling_Amelie_2020-11-04-09-14-46.csv
2nd_task_labeling_Gerda_2020-10-29-10-18-19.csv
2nd_task_labeling_Marcos_2020-11-09-04-46-17.csv
2nd_task_labeling_Renato_2020-11-09-10-31-33.csv
2nd_task_labeling_Yalemisew_2020-11-02-01-40-27.csv
3rd_task_labeling_Amelie_2020-11-04-08-14-15.csv
3rd_task_labeling_Gerda_2020-10-29-10-37-58.csv
3rd_task_labeling_Marcos_2020-11-09-04-46-17.csv
3rd_task_labeling_Renato_2020-11-09-11-51-56.csv
3rd_task_labeling_Yalemisew_2020-11-02-01-40-27.csv


In [4]:
list_dfs = []
for label_file in label_files:
    task = int(re.findall(r'[1-3]{1}', label_file)[0])
    user = re.findall(r'Amelie|Gerda|Renato|Yalemisew|Marcos', label_file)[0]
    date = re.findall(r'[0-9]{2}.[0-9]{2}', label_file)[0]
    date = f'{date}.2020'
    
    df = pd.read_csv(f'{datapath}{label_file}', names=['image name', 'class','w','h'])
    df['task'] = task
    df['user'] = user
    df['date'] = date
    df['date'] = pd.to_datetime(df['date'])
    df.drop(['w','h'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[['task', 'user', 'date', 'image name', 'class']]
    print(f'Task: {task}, User: {user}, Images Classified: {len(df)}')
    list_dfs.append(df)

Task: 1, User: Amelie, Images Classified: 392
Task: 1, User: Gerda, Images Classified: 392
Task: 1, User: Marcos, Images Classified: 392
Task: 1, User: Renato, Images Classified: 392
Task: 1, User: Yalemisew, Images Classified: 392
Task: 2, User: Amelie, Images Classified: 392
Task: 2, User: Gerda, Images Classified: 392
Task: 2, User: Marcos, Images Classified: 392
Task: 2, User: Renato, Images Classified: 392
Task: 2, User: Yalemisew, Images Classified: 392
Task: 3, User: Amelie, Images Classified: 392
Task: 3, User: Gerda, Images Classified: 442
Task: 3, User: Marcos, Images Classified: 392
Task: 3, User: Renato, Images Classified: 392
Task: 3, User: Yalemisew, Images Classified: 392


In [5]:
df_labeling = pd.concat(list_dfs)
df_labeling['id_image'] = pd.Categorical(df_labeling['image name']).codes
df_labeling = df_labeling[['task', 'user', 'date', 'image name', 'id_image', 'class']]

In [6]:
df_labeling['image_appearances'] = df_labeling.groupby('id_image')['id_image'].transform('count')
df_labeling = df_labeling[df_labeling['image_appearances'] != 1] 

In [7]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 391
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5880 non-null   int64         
 1   user               5880 non-null   object        
 2   date               5880 non-null   datetime64[ns]
 3   image name         5880 non-null   object        
 4   id_image           5880 non-null   int16         
 5   class              5880 non-null   object        
 6   image_appearances  5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 333.0+ KB


In [8]:
df_labeling.head()

,task,user,date,image name,id_image,class,image_appearances
0,1,Amelie,2020-11-20,203-resource_document_zuiderzeemuseum_B001601_...,99,[Non-Fruits],15
1,1,Amelie,2020-11-20,285-gam19649_1.jpeg,282,[Fruits],15
2,1,Amelie,2020-11-20,07101-O_389_1.jpeg,6,[Fruits],15
3,1,Amelie,2020-11-20,07101-O_927_1.jpeg,12,[Fruits],15
4,1,Amelie,2020-11-20,07101-O_957_1.jpeg,13,[Fruits],15


#### Checking the numbers of images classified after fixing names:

In [9]:
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        size = len(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image'].unique())
        print(f'Task: {task}, User: {user}, Images Classified: {size}')
    print()

Task: 1, User: Amelie, Images Classified: 392
Task: 1, User: Gerda, Images Classified: 392
Task: 1, User: Marcos, Images Classified: 392
Task: 1, User: Renato, Images Classified: 392
Task: 1, User: Yalemisew, Images Classified: 392

Task: 2, User: Amelie, Images Classified: 392
Task: 2, User: Gerda, Images Classified: 392
Task: 2, User: Marcos, Images Classified: 392
Task: 2, User: Renato, Images Classified: 392
Task: 2, User: Yalemisew, Images Classified: 392

Task: 3, User: Amelie, Images Classified: 392
Task: 3, User: Gerda, Images Classified: 392
Task: 3, User: Marcos, Images Classified: 392
Task: 3, User: Renato, Images Classified: 392
Task: 3, User: Yalemisew, Images Classified: 392



#### Checking which images were not classified:

In [10]:
images = set(df_labeling.id_image.unique())
dfs = []
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        id_list = []
        ids = images.difference(set(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image']))
        if len(ids) > 0:
            #print(f'Missing files for task: {task}, by user: {user}')
            for _id in ids:
                try:
                    id_list.append(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                    #print(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values[0])
                except:
                    pass
                    #print('--->', df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == _id), 'image name'].values)
            #print()
        dfm = pd.DataFrame({'images':id_list})
        dfm['task'] = task
        dfm['user'] = user
        dfs.append(dfm)
df_missing = pd.concat(dfs)
df_missing.to_excel(f'{datapath}/missing.xlsx')
df_missing.head()       

,images,task,user


#### Checking duplicates

In [11]:
duplicates = df_labeling.loc[df_labeling.duplicated(['task', 'user', 'image name', 'id_image'], keep=False), ['task', 'user', 'image name', 'id_image', 'class']]
duplicates.sort_values(by=['user', 'image name', 'task'], inplace=True)
duplicates.to_excel(f'{datapath}/duplicates.xlsx')
duplicates.head(30)

,task,user,image name,id_image,class


### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results

In [12]:
l1 = df_labeling.user.unique()
iter_users = list(itertools.product(l1,l1))
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [13]:
for task in df_labeling.task.unique():
    for user1,user2 in iter_users:
        classesA = df_labeling.loc[(df_labeling.user == user1) & (df_labeling.task == task),['id_image', 'class']]
        classesA.sort_values(by=['id_image'], inplace=True)

        classesB = df_labeling.loc[(df_labeling.user == user2) & (df_labeling.task == task),['id_image', 'class']]
        classesB.sort_values(by=['id_image'], inplace=True)
        
        classesAB = pd.merge(classesA, classesB, on=['id_image'])
        classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
        classesAB.drop('id_image', axis=1, inplace=True)
        classesAB.dropna(inplace=True)

        agreement = cks(classesAB['class_x'], classesAB['class_y'])
        df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'  
        df_iaa.index.name = f'Task_{task}'
    print(tabulate(df_iaa, headers='keys', tablefmt='psql'))
    print()

+-----------+-------------+-------------+-------------+-------------+-------------+
| Task_1    | Amelie      | Gerda       | Marcos      | Renato      | Yalemisew   |
|-----------+-------------+-------------+-------------+-------------+-------------|
| Amelie    | 1.000/(392) | 0.928/(392) | 0.898/(392) | 0.908/(392) | 0.898/(392) |
| Gerda     | 0.928/(392) | 1.000/(392) | 0.882/(392) | 0.907/(392) | 0.861/(392) |
| Marcos    | 0.898/(392) | 0.882/(392) | 1.000/(392) | 0.913/(392) | 0.928/(392) |
| Renato    | 0.908/(392) | 0.907/(392) | 0.913/(392) | 1.000/(392) | 0.903/(392) |
| Yalemisew | 0.898/(392) | 0.861/(392) | 0.928/(392) | 0.903/(392) | 1.000/(392) |
+-----------+-------------+-------------+-------------+-------------+-------------+

+-----------+-------------+-------------+-------------+-------------+-------------+
| Task_2    | Amelie      | Gerda       | Marcos      | Renato      | Yalemisew   |
|-----------+-------------+-------------+-------------+-------------+------

In [14]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 391
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5880 non-null   int64         
 1   user               5880 non-null   object        
 2   date               5880 non-null   datetime64[ns]
 3   image name         5880 non-null   object        
 4   id_image           5880 non-null   int16         
 5   class              5880 non-null   object        
 6   image_appearances  5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 333.0+ KB


In [15]:
df_labeling['round'] = 2

In [16]:
df_labeling.to_pickle('../data/img_labeling_2nd_round/df_labeling_round2.pkl')